# Coding Challenge: Gene Knockout Analysis with Embeddings - Task 2: Apply the In-Silico Perturbation Workflow to target ALS genes

In this notebook, we want to target previously reported genes linked to amyotrophic lateral sclerosis (ALS). 

Here we use the `helical` package for the workflow.  

In [1]:
%load_ext autoreload
%autoreload 2

## Load packages

In [1]:
import logging
import warnings

import os

import numpy as np
import pandas as pd

import anndata as ann
import scanpy as sc
import umap

from helical.models.geneformer import Geneformer, GeneformerConfig
from helical.utils import mapping
import torch

import seaborn as sns
import matplotlib.pyplot as plt


/Users/marenbuettner/miniforge3/envs/helical/lib/python3.11/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound
INFO:datasets:PyTorch version 2.6.0 available.


In [2]:
sc.logging.print_versions()

Package,Version
numpy,1.26.4
pandas,2.2.2
anndata,0.12.4
scanpy,1.11.5
umap-learn,0.5.9.post2
helical,1.4.6
torch,2.6.0
seaborn,0.13.2
matplotlib,3.10.7
Component,Info


Import custom code for perturbation and perturbation analysis.

In [ ]:
from anndata_perturbation import AnnDataPerturbationModel

## Set paths and load data

In [4]:
project_path = './'
data_dir = os.path.join(project_path, "data")
table_dir = os.path.join(project_path, "tables")
figure_dir = os.path.join(project_path, "figures")
# set scanpy figure path in addition
sc.settings.figdir = os.path.join(project_path, "figures")
# load data in anndata format
adata = sc.read(os.path.join(project_path, data_dir, "counts_combined_filtered_BA4_sALS_PN.h5ad"))

## Load model

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

Set model configuration. 

In [6]:
model_config = GeneformerConfig(model_name="gf-12L-95M-i4096", batch_size=10, device=device)
geneformer_v2 = Geneformer(model_config)

INFO:helical.models.geneformer.model:Model finished initializing.
INFO:helical.models.geneformer.model:'gf-12L-38M-i4096' model is in 'eval' mode, on device 'cpu' with embedding mode 'cell'.


## Rationale of the in-silico perturbation experiment

According to the review of [Wang et al.](https://doi.org/10.3389/fnins.2023.1170996) in Frontiers in Neuroscience in 2023, currently several genes are linked to ALS. However, only 10% of the ALS cases were associated with genetic factors. 

The following genes have been previously identified to link to ALS: *SOD1, ANXA11, ARPP21, CAV1, C21ORF2, CCNF, DNAJC7, GLT8D1, KIF5A, NEK1, SPTLC1, TIA1, TARDBP, FUS*, and *WDR7*. In addition, a recent study by [Pineda et al.](https://www.cell.com/cell/fulltext/S0092-8674(24)00234-4) in Cell 2024, identified alterations in *C9orf72* to be linked with both ALS and frontotemporal lobar degeneration (FTLD). From a pharmaceutical angle, the pathway of integrated stress response (ISR) has been found altered and led to the development of several drug candidates acting as eIF2B activators (see [Marlin et al.](https://bpspubs.onlinelibrary.wiley.com/doi/10.1111/bph.16260) and [Flores et al.](https://www.nature.com/articles/s41467-025-63031-y)) to reduce the ISR activity and prevent neurodegeneration.  

To investigate the impact of perturbations in the given data set, we may explore the following options:
1. Run single gene perturbations
2. Run combinations of gene perturbations
3. Run pathway perturbations

The computationally cleanest approach would be to investigate single gene perturbations, which allowed us to identify the genes with strongest impact on healthy cells and identify the ones which generate a perturbation that is resembling the disease phenotype the most. From the biological perspective, option 3 would make the most sense, e.g. use the combination of genes involved in RNA metabolism (*TIA1, TARDBP, FUS*, and *C9orf72*), but which would not directly allow to identify a specific target. However, a combination of single gene perturbations and pathway perturbations could provide useful context in the strength of individual factors vs combinatorial perturbations. 

In the following, we will perform a knockup experiment of genes involved in RNA metabolism (*TIA1, TARDBP, FUS*, and *C9orf72*) and perform a pathway perturbation as knockup experiment of these genes in addition. We chose the knockup scenario because mutations in *TARDBP* reportedly led to overexpression and accumulation of its gene product TDP-43, an  RNA binding protein (RBP) involved in RNA processing (see [Flores et al.](https://www.nature.com/articles/s41467-025-63031-y)).  

## Process data and generate embeddings

We noted previously that the `anndata` object does not encompass any data processing except for initial cell and gene filtering, or low-dimensional embeddings. 

Here we use the `helical` wrapper function for the `geneformer` Transcriptome Tokenizer to process the data. 

For the perturbation modeling of ALS, we subset the data to the pathologically normal `PN` condition, which the healthy control state in contrast to the `ALS` condition. We do not further subset to any cell types because we want to explore if there are cell-type specific effects of the perturbation. 

**Comment:** To keep the compute time on my MacBook in a reasonable time frame (i.e. less than 70 h to compute an embedding), I subset the data to 1,000 observations.

In [9]:
adata_pn = adata[adata.obs['Condition'] == 'PN'].copy()

In [10]:
adata_1k = sc.pp.sample(adata_pn, n=1000, copy = True)

In [11]:
# Initialize (tokenizes and embeds original data once)
perturb_model = AnnDataPerturbationModel(
    foundation_model=geneformer_v2,
    adata=adata_1k,
    normalize_embeddings=True
)

INFO:helical.models.geneformer.model:Processing data for Geneformer.


Tokenizing original data...


INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/marenbuettner/Library/Caches/pyensembl/GRCh38/ensembl110/Homo_sapiens.GRCh38.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/marenbuettner/Library/Caches/pyensembl/GRCh38/ensembl110/Homo_sapiens.GRCh38.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/marenbuettner/Library/Caches/pyensembl/GRCh38/ensembl110/Homo_sapiens.GRCh38.pep.all.fa.gz.pickle
INFO:helical.utils.mapping:Mapped 22830 genes to Ensembl IDs from a total of 22832 genes.
INFO:helical.models.geneformer.geneformer_tokenizer:AnnData object with n_obs × n_vars = 1000 × 22832
    obs: 'Sample_ID', 'Donor', 'Region', 'Sex', 'Condition', 'Group', 'C9_pos', 'CellClass', 'CellType', 'SubType', 'full_label', 'DGE_Group', 'Bakken_M1', 'data_merge_id', 'data_sample_id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'Cellstates_LVL1', 'Cellstates_LVL

Computing original embeddings...


  0%|          | 0/100 [00:00<?, ?it/s]

INFO:helical.models.geneformer.model:Finished getting embeddings.


Original embeddings shape: (1000, 512)


## Simulate a knockup

Here we set up the knockup experiment of genes involved in the RNA metabolism, firstly as single gene perturbation and secondly in combination of all genes for context.  

In [13]:
genes_to_perturb = ['TIA1', 'TARDBP', 'FUS', 'C9orf72']

In [14]:
# Perturb each gene in the list individually
results_batch = perturb_model.batch_perturbation(
    gene_list=genes_to_perturb,
    perturbation_type='overexpression',
    fold_change=2.0
)


[1/4] Processing TIA1...
Creating perturbed AnnData (overexpression: TIA1, FC=2.0)...


INFO:helical.models.geneformer.model:Processing data for Geneformer.


Tokenizing perturbed data...


INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/marenbuettner/Library/Caches/pyensembl/GRCh38/ensembl110/Homo_sapiens.GRCh38.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/marenbuettner/Library/Caches/pyensembl/GRCh38/ensembl110/Homo_sapiens.GRCh38.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/marenbuettner/Library/Caches/pyensembl/GRCh38/ensembl110/Homo_sapiens.GRCh38.pep.all.fa.gz.pickle
INFO:helical.utils.mapping:Mapped 22830 genes to Ensembl IDs from a total of 22832 genes.
INFO:helical.models.geneformer.geneformer_tokenizer:AnnData object with n_obs × n_vars = 1000 × 22832
    obs: 'Sample_ID', 'Donor', 'Region', 'Sex', 'Condition', 'Group', 'C9_pos', 'CellClass', 'CellType', 'SubType', 'full_label', 'DGE_Group', 'Bakken_M1', 'data_merge_id', 'data_sample_id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'Cellstates_LVL1', 'Cellstates_LVL

Computing perturbed embeddings...


...


[4/4] Processing C9orf72...
Creating perturbed AnnData (overexpression: C9orf72, FC=2.0)...
Tokenizing perturbed data...


INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/marenbuettner/Library/Caches/pyensembl/GRCh38/ensembl110/Homo_sapiens.GRCh38.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/marenbuettner/Library/Caches/pyensembl/GRCh38/ensembl110/Homo_sapiens.GRCh38.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/marenbuettner/Library/Caches/pyensembl/GRCh38/ensembl110/Homo_sapiens.GRCh38.pep.all.fa.gz.pickle
INFO:helical.utils.mapping:Mapped 22830 genes to Ensembl IDs from a total of 22832 genes.
INFO:helical.models.geneformer.geneformer_tokenizer:AnnData object with n_obs × n_vars = 1000 × 22832
    obs: 'Sample_ID', 'Donor', 'Region', 'Sex', 'Condition', 'Group', 'C9_pos', 'CellClass', 'CellType', 'SubType', 'full_label', 'DGE_Group', 'Bakken_M1', 'data_merge_id', 'data_sample_id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'Cellstates_LVL1', 'Cellstates_LVL

Computing perturbed embeddings...


  0%|          | 0/100 [00:00<?, ?it/s]

INFO:helical.models.geneformer.model:Finished getting embeddings.


In [15]:
# Gene overexpression simulation for the entire pathway
result_pathway = perturb_model.overexpression(
    genes=genes_to_perturb,
    fold_change=2.0
)

INFO:helical.models.geneformer.model:Processing data for Geneformer.


Creating perturbed AnnData (overexpression: TIA1, TARDBP, FUS, C9orf72, FC=2.0)...
Tokenizing perturbed data...


INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/marenbuettner/Library/Caches/pyensembl/GRCh38/ensembl110/Homo_sapiens.GRCh38.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/marenbuettner/Library/Caches/pyensembl/GRCh38/ensembl110/Homo_sapiens.GRCh38.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/marenbuettner/Library/Caches/pyensembl/GRCh38/ensembl110/Homo_sapiens.GRCh38.pep.all.fa.gz.pickle
INFO:helical.utils.mapping:Mapped 22830 genes to Ensembl IDs from a total of 22832 genes.
INFO:helical.models.geneformer.geneformer_tokenizer:AnnData object with n_obs × n_vars = 1000 × 22832
    obs: 'Sample_ID', 'Donor', 'Region', 'Sex', 'Condition', 'Group', 'C9_pos', 'CellClass', 'CellType', 'SubType', 'full_label', 'DGE_Group', 'Bakken_M1', 'data_merge_id', 'data_sample_id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'Cellstates_LVL1', 'Cellstates_LVL

Computing perturbed embeddings...


  0%|          | 0/100 [00:00<?, ?it/s]

INFO:helical.models.geneformer.model:Finished getting embeddings.


## Save results and embeddings to file

In [16]:
import pickle

In [19]:
with open(os.path.join(data_dir, 'perturbmodel_1k_02.pkl'), 'wb') as handle:
    pickle.dump(perturb_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
with open(os.path.join(data_dir, 'perturbmodel_1k_batch_knockup.pkl'), 'wb') as handle:
    pickle.dump(results_batch, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
with open(os.path.join(data_dir, 'perturbmodel_1k_pathway_knockup.pkl'), 'wb') as handle:
    pickle.dump(result_pathway, handle, protocol=pickle.HIGHEST_PROTOCOL)

End of the workflow.